In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup #as soup
import time
import json

from pathlib import Path  
import os

versions of the core libraries used
- pandas version: 1.0.1

- BeautifulSoup version: 4.8.2

- requests version: 2.22.0

- python version: 3.7.3

In [2]:
# tested only on windows, fetch the username folder, assist in saving location
# first = len("c:\\users\\")
# print(os.path.abspath(os.getcwd()))
# pwd = os.path.abspath(os.getcwd())
# second = pwd[first:len(pwd)]
# third = second.find("\\")
# pc_username_folder = second[0:third]
# print("username: "+ pc_username_folder)

# save_loc =fr'C:\Users\{pc_username_folder}\Desktop\pharma_ida.csv'

In [3]:
#manually type save location
save_loc =fr'C:\Users\rsimon\Desktop\pharma_ida.csv'

In [4]:
#This URL is the page you actually want to pull down with requests.
initial_page = 'https://idbop.mylicense.com/verification/Search.aspx'
searchPage = initial_page

In [5]:
# format the fetch list into  columns.
def list_format(list_name, step):
    return [list_name[i::step] for i in range(step)]
#print(list_format(data,4))

In [6]:
def fetch_next_page(page):
    payload_next_page = {
    "CurrentPageIndex":page,
    "__EVENTTARGET":"datagrid_results$_ctl44$_ctl1",
    "__EVENTARGUMENT":"",
    "__VIEWSTATEGENERATOR":viewStateGenerator,
    "__VIEWSTATE":viewState,
    "__EVENTVALIDATION":eventValidation}
    nextpage="https://idbop.mylicense.com/verification/SearchResults.aspx"
    result_np=req_sess.post(nextpage,headers=headers1,data=payload_next_page)
    soup1=BeautifulSoup(result_np.content, 'html.parser')
    td_as_base1 = soup1.find_all('td') 
    data2 = []
    for cell in td_as_base1:
            if cell.text=="":
                #print("empty string skipped")
                continue
            #if cell.text.startswith('\n'):
            if '\\' in  r"%r" % cell.text:
                #print("slash skipped")
                continue
            insertdata = cell.text.strip()
            data2.append(insertdata)
    return data2

In [7]:
def count_pages():
    pagination=BeautifulSoup(result.content, 'html.parser')
    paginationtags = pagination.find_all('a') # all trs
    pages = [tag.text.strip() for tag in paginationtags if tag.text.isnumeric()]
    return pages

In [8]:
def remove_these(thisList):
    try:
        thisList.remove("Search")
        thisList.remove("Search Results")
        print("Removed exess texts")
    except:
        print("keywords removed/not present")

In [9]:
############# SEQUENCE 1 - the initial page needs to be loaded first to acquire the viewstate to be used in the search page, then the pagination

In [10]:
print("Loading the initial page")
req_sess=requests.Session()
r=req_sess.get(initial_page)
time.sleep(5)
soup=BeautifulSoup(r.content, 'html.parser')

Loading the initial page


In [11]:
# eventTarget=soup.find(id="__EVENTTARGET")['value']
#eventArguement=soup.find(id="__EVENTARGUMENT")['value']
viewState=soup.find(id="__VIEWSTATE")['value']
viewStateGenerator=soup.find(id="__VIEWSTATEGENERATOR")['value']
eventValidation=soup.find(id="__EVENTVALIDATION")['value']

In [12]:
# 
license_type_name = "Pharmacist"
lastName ="L*"



In [13]:
### search fields , Please uncomment after testing



# lastName = input("Enter Last Name")
# first_name  = input('Enter First Name')
# license_type_name =input("Enter License Type")
# license_type_name  = input('Enter License Type')
# last_name  = input('Enter Last Name')
# addr_city  = input('Enter City')
# license_no  = input('Enter license no')
# addr_state  = input('Enter State')
# license_status_name  = input('Enter license status')
# addr_county  = input('Enter Country')
# addr_zipcode  = input('Enter Zip')

In [14]:
payload = {
    "__EVENTTARGET":"",
    "__EVENTARGUMENT":"",
    "__LASTFOCUS":"",
    "__VIEWSTATEGENERATOR":viewStateGenerator,
    "__VIEWSTATE":viewState,
    "__EVENTVALIDATION":eventValidation,
    "t_web_lookup__first_name":"",
    "t_web_lookup__license_type_name":license_type_name,
    "t_web_lookup__last_name":lastName,
    "t_web_lookup__addr_city":"",
    "t_web_lookup__license_no":"",
    "t_web_lookup__addr_state":"",
    "t_web_lookup__license_status_name":"",
    "t_web_lookup__addr_county":"",
    "t_web_lookup__addr_zipcode":"",
    "sch_button":"Search"
}

In [15]:
headers1 = {
    'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Accept-Encoding':'gzip, deflate, br',
    'Connection':'keep-alive',
    'Cache-Control':'max-age=0',
    'Content-Length': '2853',
    'Content-Type':'application/x-www-form-urlencoded; charset=UTF-8',
    'Host':'idbop.mylicense.com',
    'Origin':'https://idbop.mylicense.com',
    'Referer': 'https://idbop.mylicense.com/verification/Search.aspx',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36 Edg/108.0.1462.54',
    'X-Requested-With':'XMLHttpRequest'
}

In [16]:
print("submitting inital requet to page")
result=req_sess.post(searchPage,headers=headers1,data=payload)

submitting inital requet to page


In [17]:
# for fetching the page count
soup=BeautifulSoup(result.text, 'html.parser')

In [18]:
pageCount =count_pages()

In [19]:
pageCount

['2', '3', '4', '5', '6', '7', '8', '9']

In [20]:
# iterate fetching of all pages
all_list =[]
for page in range(len(pageCount)+1):
    print("fetching page "+str(page))
    fetchedData = fetch_next_page(page)
    # remove the exces texts constanly found
    remove_these(fetchedData)
    all_list.append(fetchedData)
    print("page "+str(page)+" fetch done")
    # a bit of a pause before the next query
    time.sleep(2)

fetching page 0
Removed exess texts
page 0 fetch done
fetching page 1
Removed exess texts
page 1 fetch done
fetching page 2
Removed exess texts
page 2 fetch done
fetching page 3
Removed exess texts
page 3 fetch done
fetching page 4
Removed exess texts
page 4 fetch done
fetching page 5
Removed exess texts
page 5 fetch done
fetching page 6
Removed exess texts
page 6 fetch done
fetching page 7
Removed exess texts
page 7 fetch done
fetching page 8
Removed exess texts
page 8 fetch done


In [21]:
# consolitdate al in one list
finalList =[]
for item in all_list:
    for cells in  item:
        finalList.append(cells)

In [22]:
len(finalList)

1360

In [23]:
print("formatting fetched list")
formattedFinalList = list_format(finalList,4)

formatting fetched list


In [24]:
# resconstruct the table
print("constructing table")
Name = formattedFinalList[0]
License_NO = formattedFinalList[1]
Licence_type =   formattedFinalList[2]
Status =  formattedFinalList[3]
phamacist_list_dict = {'Name': Name, 'License_NO': License_NO, 'Licence_type': Licence_type,"Status":Status}
final_list =  pd.DataFrame(phamacist_list_dict)
#len(final_list)

constructing table


In [25]:
final_list

,Name,License_NO,Licence_type,Status
0,"La, Abbie Anh-Buu",P8571,Pharmacist,Expired
1,"LABELLA, JAMES PAUL",P7148,Pharmacist,Active
2,"LABREL, KEATEN PATRICK",P7351,Pharmacist,Expired
3,"LACEY, ROY ARTHUR",P7222,Pharmacist,Expired
4,"LACHELT, JEFF A",P5376,Pharmacist,Active
...,...,...,...,...
335,"LYONS, MARY J",P5213,Pharmacist,Active
336,"LYONS, TOM",P3049,Pharmacist,Expired
337,"LYSTRUP, ALEX JAMES, PHARMD",P8065,Pharmacist,Active
338,"LYTLE, RYAN F",P6435,Pharmacist,Active


In [27]:
print("saving file")
filepath = Path(save_loc)  
filepath.parent.mkdir(parents=True, exist_ok=True)  
final_list.to_csv(filepath)

saving file


PermissionError: [WinError 5] Access is denied: 'C:\\Users\\rsimon'

In [ ]:
print("scrape completed")